**Text Generation using Bidirectional LSTM**

A simple implementation of text generation using a deep LSTM NN, inspired by a tutorial I found at [link](https://www.machinelearningmastery.com)

In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# Load input file
input = "/content/drive/My Drive/LSTM/TheOneWithSeason5.txt"
raw = open(input, 'r', encoding='utf-8').read()
raw = raw.lower()

# Map
chars = sorted(list(set(raw)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [0]:
## Code for Data Loading and Displaying results taken from: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

# Summarize
n_chars = len(raw)
n_vocab = len(chars)
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw[i:i + seq_length]
	seq_out = raw[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
print("Total Patterns: ", n_patterns)

In [0]:
# Define Model
model = Sequential()
model.add(Bidirectional(LSTM(128, activation="relu"),input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

In [0]:
import sys

## Put file Name for Weights for model here 
filename = "/content/weights-improvement-20-2.4410.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

int_to_char = dict((i, c) for i, c in enumerate(chars))

start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]

print("Seed:")
print("", ''.join([int_to_char[value] for value in pattern]))

for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."